In [1]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
with open("chatbot_dataset.json", "r") as f:
    data = json.load(f)

intents = data["intents"]

texts = []
labels = []

for intent in intents:
    tag = intent["tag"]
    for pattern in intent["patterns"]:
        texts.append(pattern.strip().lower())
        labels.append(tag)

In [3]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

In [4]:
tokenizer = Tokenizer(num_words=3000, oov_token="<unk>")
tokenizer.fit_on_texts(texts)

X = tokenizer.texts_to_sequences(texts)

max_len = 6
X = pad_sequences(X, maxlen=max_len, padding="post")

vocab_size = min(3000, len(tokenizer.word_index) + 1)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42,
    stratify=y
)

In [6]:
inputs = layers.Input(shape=(max_len,))

x = layers.Embedding(
    vocab_size,
    64
)(inputs)

x = layers.Conv1D(128, 3, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 6, 64)          │        19,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 4, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 14)             │           910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,134 (207.55 KB)

 Trainable params: 53,134 (207.55 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "best_model.keras",
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False
)

In [8]:
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, checkpoint]
)

Epoch 1/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.1660 - loss: 2.5885 - val_accuracy: 0.6984 - val_loss: 1.9641
Epoch 2/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6992 - loss: 1.5735 - val_accuracy: 0.9810 - val_loss: 0.2802
Epoch 3/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9629 - loss: 0.2800 - val_accuracy: 0.9873 - val_loss: 0.0741
Epoch 4/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9919 - loss: 0.0861 - val_accuracy: 0.9873 - val_loss: 0.0416
Epoch 5/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9975 - loss: 0.0382 - val_accuracy: 0.9873 - val_loss: 0.0353
Epoch 6/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9992 - loss: 0.0217 - val_accuracy: 0.9873 - val_loss: 0.0379
Epoch 7/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9986 - loss: 0.0182 - val_accuracy: 0.9873 - val_loss: 0.0421
Epoch 8/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0118 - val_accuracy: 0

In [9]:
def predict_intent(text):
    text = text.lower().strip()
    seq = tokenizer.texts_to_sequences([text])
    seq = pad_sequences(seq, maxlen=max_len, padding="post")

    pred = model.predict(seq)[0]
    tag = label_encoder.inverse_transform([np.argmax(pred)])[0]
    confidence = float(np.max(pred))

    return tag, confidence

In [11]:
query = "hi i want to here a joke"
tag, conf = predict_intent(query)
print("Intent:", tag)
print("Confidence:", conf)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Intent: small_talk
Confidence: 0.9645305275917053
